In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'
!kaggle datasets download -d enesumcu/car-and-truck
#!kaggle datasets download -d gaborfodor/keras-pretrained-models

 99% 166M/167M [00:12<00:00, 14.8MB/s]
100% 167M/167M [00:12<00:00, 14.0MB/s]


In [2]:
!unzip car-and-truck.zip
#!unzip keras-pretrained-models.zip

Archive:  car-and-truck.zip
  inflating: Datasets/car/100171.jpg  
  inflating: Datasets/car/102685.jpg  
  inflating: Datasets/car/103773.jpg  
  inflating: Datasets/car/111104.jpg  
  inflating: Datasets/car/115988.jpg  
  inflating: Datasets/car/119092.jpg  
  inflating: Datasets/car/128887.jpg  
  inflating: Datasets/car/130726.jpg  
  inflating: Datasets/car/13193.jpg  
  inflating: Datasets/car/137920.jpg  
  inflating: Datasets/car/139345.jpg  
  inflating: Datasets/car/141101.jpg  
  inflating: Datasets/car/143302.jpg  
  inflating: Datasets/car/145185.jpg  
  inflating: Datasets/car/145195.jpg  
  inflating: Datasets/car/145512.jpg  
  inflating: Datasets/car/146072.jpg  
  inflating: Datasets/car/153019.jpg  
  inflating: Datasets/car/155229.jpg  
  inflating: Datasets/car/155509.jpg  
  inflating: Datasets/car/155828.jpg  
  inflating: Datasets/car/156277.jpg  
  inflating: Datasets/car/158493.jpg  
  inflating: Datasets/car/16299.jpg  
  inflating: Datasets/car/164486.jpg  

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf

from IPython.display import Image, display

import matplotlib.pyplot as plt
import seaborn as sns

import cv2
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from keras.layers import Activation,Dense,Flatten,BatchNormalization,Conv2D,MaxPool2D,Dropout,Input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras import backend as K
from keras.applications import VGG16

import warnings
warnings.filterwarnings('ignore')

In [13]:
!pip install fastai==1.0.61

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 239 kB 31.6 MB/s 
     |████████████████████████████████| 355 kB 66.2 MB/s 
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19190 sha256=aabe8d1eb554a8dd27b2cbd5e44d58566d1f1b8e59eb3825bce318fb150315d8
  Stored in directory: /root/.cache/pip/wheels/df/99/da/c34f202dc8fd1dffd35e0ecf1a7d7f8374ca05fbcbaf974b83
Successfully built nvidia-ml-py3
  Attempting uninstall: fastai
    Found existing installation: fastai 2.7.9
    Uninstalling fastai-2.7.9:
      Successfully uninstalled fastai-2.7.9


In [1]:
import os, cv2
import numpy as np
import tensorflow as tf
from tensorflow.python import keras

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, AvgPool2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
#from keras.preprocessing.image import img_to_array, array_to_img, load_img
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
from fastai.imports import *
from fastai import *
from fastai.vision import *
from torchvision.models import *

import numpy as np


In [3]:
path = '/content/datasets/Datasets'

In [4]:
for c in ["truck", "car"]:
  print(c)
  verify_images(path+"/"+c, delete=True, max_size=500)

truck


car


cannot identify image file <_io.BufferedReader name='/content/datasets/Datasets/car/181535.jpg'>


In [5]:
cars = (len(os.listdir(path+"/car")))
trucks = (len(os.listdir(path+"/truck")))

In [6]:
def data_frame(data_dir):
    a=[]
    for i in os.listdir(data_dir):
        for k,j in enumerate(os.listdir(data_dir+'/'+i)):
            a.append((f'/{i}/{j}',i))
    return pd.DataFrame(a,columns=['filename','class'])

In [7]:
df = data_frame(path)
print(df.shape)
df.head() 

(788, 2)


,filename,class
0,/car/556310.jpg,car
1,/car/images (7).jpg,car
2,/car/188317.jpg,car
3,/car/636308.jpg,car
4,/car/600109.jpg,car


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
        df['filename'], df['class'],stratify=df['class'], test_size=0.1)

In [10]:
train = pd.DataFrame(zip(X_train.values, y_train.values), columns=['filename', 'class'], index=None)
test = pd.DataFrame(zip(X_test.values, y_test.values), columns=['filename', 'class'], index=None)
test['filename'] = path + test['filename']

In [11]:
data = ImageDataBunch.from_df(path, train, 
                                  valid_pct=0.2, 
                                  bs=32, 
                                  size=256).normalize(imagenet_stats)

In [12]:
data

ImageDataBunch;

Train: LabelList (568 items)
x: ImageList
Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256)
y: CategoryList
truck,truck,car,car,truck
Path: /content/datasets/Datasets;

Valid: LabelList (141 items)
x: ImageList
Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256)
y: CategoryList
car,car,car,car,truck
Path: /content/datasets/Datasets;

Test: None

# ResNet-50

In [13]:
learn_resnet50 = cnn_learner(data, models.resnet50, metrics=accuracy)
learn_resnet50.fit(epochs = 10)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,accuracy,time
0,0.627686,0.521721,0.900709,00:14
1,0.382830,0.337376,0.900709,00:07
2,0.251565,0.264955,0.929078,00:07
3,0.180283,0.340107,0.907801,00:07
4,0.131028,0.293278,0.914894,00:08
5,0.093536,0.287609,0.914894,00:09
6,0.066010,0.284850,0.950355,00:07
7,0.048677,0.283885,0.950355,00:07
8,0.036963,0.297546,0.936170,00:07
9,0.029258,0.373897,0.929078,00:07


# ResNet-101

In [14]:
learn_resnet101 = cnn_learner(data, models.resnet101, metrics=accuracy)
learn_resnet101.fit(epochs = 10)

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,accuracy,time
0,0.644255,0.630973,0.851064,00:10
1,0.365438,0.270526,0.921986,00:10
2,0.230087,0.298903,0.900709,00:10
3,0.166358,0.296060,0.907801,00:10
4,0.123312,0.360167,0.900709,00:10
5,0.089950,0.303864,0.900709,00:10
6,0.066271,0.281932,0.921986,00:11
7,0.047867,0.253072,0.929078,00:10
8,0.044114,0.221309,0.929078,00:10
9,0.034619,0.291524,0.921986,00:10


#ResNet-152

In [15]:
learn_resnet152 = cnn_learner(data, models.resnet152, metrics=accuracy)
learn_resnet152.fit(epochs = 10)

Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,accuracy,time
0,0.547554,0.361800,0.914894,00:14
1,0.296231,0.228352,0.929078,00:14
2,0.201685,0.216135,0.936170,00:14
3,0.144806,0.181569,0.943262,00:14
4,0.101548,0.214153,0.950355,00:14
5,0.083972,0.133818,0.957447,00:14
6,0.058401,0.157206,0.936170,00:14
7,0.043509,0.143057,0.936170,00:14
8,0.033262,0.242705,0.929078,00:14
9,0.033095,0.257626,0.936170,00:14


#DenseNet-121

In [16]:
learn_densenet121 = cnn_learner(data, models.densenet121, metrics=accuracy)
learn_densenet121.fit(epochs = 10)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,accuracy,time
0,0.592399,0.353270,0.900709,00:09
1,0.386686,0.284721,0.929078,00:07
2,0.235255,0.297263,0.914894,00:08
3,0.162082,0.286878,0.907801,00:08
4,0.131037,0.299535,0.907801,00:07
5,0.103838,0.356725,0.900709,00:07
6,0.073080,0.286691,0.907801,00:07
7,0.063711,0.307247,0.907801,00:07
8,0.051429,0.464912,0.900709,00:08
9,0.045565,0.391614,0.893617,00:08


# DenseNet-161

In [17]:
learn_densenet161 = cnn_learner(data, models.densenet161, metrics=accuracy)
learn_densenet161.fit(epochs = 10)

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth


  0%|          | 0.00/110M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,accuracy,time
0,0.525843,0.241256,0.936170,00:21
1,0.286739,0.206668,0.936170,00:14
2,0.196007,0.159078,0.943262,00:14
3,0.135888,0.162669,0.950355,00:14
4,0.091127,0.110475,0.950355,00:14
5,0.064123,0.113313,0.950355,00:14
6,0.044022,0.097421,0.964539,00:14
7,0.031937,0.086708,0.964539,00:14
8,0.028622,0.124412,0.957447,00:14
9,0.024671,0.123435,0.936170,00:14


#Predictions

In [18]:
def predict_car_or_truck(model_learn, filepath):
    img = open_image(filepath)
    return model_learn.predict(img)[0]

In [19]:
test['resnet50'] = test['filename'].apply(lambda x: predict_car_or_truck(learn_resnet50, x))
test['resnet50'] = test['resnet50'].astype('str')
test.head()

,filename,class,resnet50
0,/content/datasets/Datasets/truck/747398733.jpg,truck,truck
1,/content/datasets/Datasets/car/842548.jpg,car,car
2,/content/datasets/Datasets/car/312250.jpg,car,car
3,/content/datasets/Datasets/car/16299.jpg,car,truck
4,/content/datasets/Datasets/car/4560.jpg,car,car


In [20]:
test['resnet101'] = test['filename'].apply(lambda x: predict_car_or_truck(learn_resnet101, x))
test['resnet101'] = test['resnet101'].astype('str')
test.head()

,filename,class,resnet50,resnet101
0,/content/datasets/Datasets/truck/747398733.jpg,truck,truck,truck
1,/content/datasets/Datasets/car/842548.jpg,car,car,car
2,/content/datasets/Datasets/car/312250.jpg,car,car,car
3,/content/datasets/Datasets/car/16299.jpg,car,truck,truck
4,/content/datasets/Datasets/car/4560.jpg,car,car,car


In [21]:
test['resnet152'] = test['filename'].apply(lambda x: predict_car_or_truck(learn_resnet152, x))
test['resnet152'] = test['resnet152'].astype('str')
test.head()

,filename,class,resnet50,resnet101,resnet152
0,/content/datasets/Datasets/truck/747398733.jpg,truck,truck,truck,truck
1,/content/datasets/Datasets/car/842548.jpg,car,car,car,car
2,/content/datasets/Datasets/car/312250.jpg,car,car,car,car
3,/content/datasets/Datasets/car/16299.jpg,car,truck,truck,truck
4,/content/datasets/Datasets/car/4560.jpg,car,car,car,car


In [22]:
test['densenet121'] = test['filename'].apply(lambda x: predict_car_or_truck(learn_densenet121, x))
test['densenet121'] = test['densenet121'].astype('str')
test.head()

,filename,class,resnet50,resnet101,resnet152,densenet121
0,/content/datasets/Datasets/truck/747398733.jpg,truck,truck,truck,truck,truck
1,/content/datasets/Datasets/car/842548.jpg,car,car,car,car,car
2,/content/datasets/Datasets/car/312250.jpg,car,car,car,car,car
3,/content/datasets/Datasets/car/16299.jpg,car,truck,truck,truck,truck
4,/content/datasets/Datasets/car/4560.jpg,car,car,car,car,car


In [23]:
test['densenet161'] = test['filename'].apply(lambda x: predict_car_or_truck(learn_densenet161, x))
test['densenet161'] = test['densenet161'].astype('str')
test.head()

,filename,class,resnet50,resnet101,resnet152,densenet121,densenet161
0,/content/datasets/Datasets/truck/747398733.jpg,truck,truck,truck,truck,truck,truck
1,/content/datasets/Datasets/car/842548.jpg,car,car,car,car,car,car
2,/content/datasets/Datasets/car/312250.jpg,car,car,car,car,car,car
3,/content/datasets/Datasets/car/16299.jpg,car,truck,truck,truck,truck,truck
4,/content/datasets/Datasets/car/4560.jpg,car,car,car,car,car,car


#Evaluation of models built

In [24]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [25]:
models = ['resnet50', 'resnet101', 'resnet152',
       'densenet121', 'densenet161']
scores = dict()
for model in models:
  scores[model] = [accuracy_score(test['class'], test[model])]

In [26]:
print("Accuracy scores:")
pd.DataFrame(scores)

Accuracy scores:


,resnet50,resnet101,resnet152,densenet121,densenet161
0,0.873418,0.936709,0.936709,0.898734,0.898734


In [27]:
confusion_matrix(test['class'], test['resnet50'])

array([[35,  4],
       [ 6, 34]])

In [28]:
confusion_matrix(test['class'], test['resnet101'])

array([[37,  2],
       [ 3, 37]])

In [29]:
confusion_matrix(test['class'], test['resnet152'])

array([[37,  2],
       [ 3, 37]])

In [30]:
confusion_matrix(test['class'], test['densenet121'])

array([[36,  3],
       [ 5, 35]])

In [31]:
confusion_matrix(test['class'], test['densenet161'])

array([[36,  3],
       [ 5, 35]])